In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [14]:
dtf=['month','week','day','dayofweek']

In [15]:
def Vectorize_client(path, feathers):   
    full_client_vec_dt=pd.DataFrame()    
    table = pq.read_table(path,columns=['client','timestamp'])
    obraz=table.to_pandas()
    for feath in feathers:
        obraz[feath] = getattr(obraz['timestamp'].dt, feath)
    obraz.drop('timestamp', axis=1, inplace=True)
    clients=obraz['client'].unique()
    clfeatval= pd.DataFrame()
    for client in clients:
        one_client=obraz.loc[obraz['client'] == client]   
        cli_val=[]
        for feather in feathers:#print(one_client[feather].value_counts()) 
            client_fet=[]
            list_veckor=pd.read_csv((feather+'_vec.csv'),index_col = 0)
            for i in one_client.index:
                element4finde=one_client.loc[i,feather]
                client_fet.append(list_veckor.loc[element4finde])
            cf=np.array(client_fet)
            cff=np.reshape(cf, (-1,len(list_veckor.loc[element4finde].values)))        
            cff=np.sum(cff, axis=0)
            cli_val.extend(cff)
        clfeatval[client] = cli_val
        #print(clfeatval.T)
    full_clients_vec=pd.concat([full_client_vec_dt,clfeatval.T], axis=1)
    #print(feather,full_clients_vec.shape)
    return full_clients_vec

In [16]:
pathes=['part-00000.parquet','part-00001.parquet','part-00002.parquet','part-00003.parquet','part-00004.parquet','part-00005.parquet','part-00006.parquet','part-00007.parquet','part-00008.parquet','part-00009.parquet']

In [18]:
num=0
for p in pathes:    
    g=Vectorize_client(p, dtf)
    g.to_csv(('dtf_vector'+str(num)+'.csv'))
    print(g.shape, '____#',str(num),'____Ok')
    num+=1

<ipython-input-15-9770fcc8a863>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  obraz[feath] = getattr(obraz['timestamp'].dt, feath)


(8211, 18) ____# 0 ____Ok


KeyboardInterrupt: 

In [12]:
pd.read_csv(('year_vec.csv'),index_col = 0)

,0
2020,1
